In [1]:
import pandas as pd
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, LeakyReLU, PReLU, LocallyConnected2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras import backend as K
import tensorflow as tf

from scipy.ndimage import rotate

from random import uniform
from numpy.random import choice

from matplotlib import pyplot as plt
%matplotlib inline

from randomized_relu import randomized_relu
from drop_activation import drop_activation
from keras.utils.generic_utils import get_custom_objects

get_custom_objects().update({'randomized_relu': Activation(randomized_relu)})
get_custom_objects().update({'drop_activation': Activation(drop_activation)})

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define mirror functions
def mirror_X(input_X):
    return input_X.reshape(-1,96,96)[:, :, list(range(95, -1, -1))].reshape(-1, 96*96)
    
def mirror_y(input_y, n_point):
    output_y = input_y.copy()
    # y coordinate stays the same, x coordinate is 96-x
    output_y[:,list(range(0,n_point-1,2))] = 95 - input_y[:,list(range(0,n_point-1,2))]
    return output_y

In [3]:
# mirror function to transform X
def rotate_X(input_X, degree):
    
    # create an array of the same size
    output_X = np.zeros((input_X.shape), dtype=np.float64)
    
    # for each image, use sklearn's rotate function to rotate image
    # this somehow changes pixel values, cap value between 0 and 255
    for i in range(output_X.shape[0]):
        output_X[i] = np.clip(rotate(input_X.reshape(-1,96,96)[i], degree, reshape=False, mode='constant', \
                                       cval=150).reshape(96*96), 0, 255)  
    return output_X
        
    
# mirror function to transform y
def rotate_y(input_y, degree, n_point):
    
    # create an array of same size
    output_y = np.zeros((input_y.shape), dtype=np.float64)
    
    # define rotation angle and center of rotation
    theta = np.radians(degree)
    center_x = 95/2
    center_y = 95/2

    for i in range(output_y.shape[0]):
        
        # obtain x and y coordinates from dataset
        x = input_y[i][np.arange(0,n_point-1,2)]
        y = input_y[i][np.arange(1,n_point,2)]
        
        # use rotation matrix to rotate coordinates around center
        x2 = np.cos(theta) * (x - center_x) + np.sin(theta) * (y - center_y) + center_x
        y2 = -np.sin(theta) * (x - center_x) + np.cos(theta) * (y - center_y) + center_y
        
        # put x and y back in original shape
        output_y[i] = np.vstack((x2,y2)).transpose().flatten()
        
    return output_y

In [4]:
# define function - only need to transform X
def reduce_contrast(input_X, weight):
    return (weight * input_X) + (1 - weight) * input_X.mean()

In [6]:
# prep data
def prep_data(X, y, points, val_pct):
    
    # model with specific points 
    y_model = y[:, points]

    # get index of non-missing labels
    index = ~np.isnan(y_model).any(axis=1)

    # filter out missing labels on training and validation data
    X_model = X[index]
    y_model = y_model[index]
    
    # create training and validation set (80/20 split)
    X_train, X_val = X_model[:int(X_model.shape[0]*(1-val_pct))], X_model[int(X_model.shape[0]*(1-val_pct)):]
    y_train, y_val = y_model[:int(X_model.shape[0]*(1-val_pct))], y_model[int(X_model.shape[0]*(1-val_pct)):]

    # training data transformation
    degree1 = -10
    degree2 = 10
    n_point = len(points)
    percent = 0.9

    X_rotated = np.zeros((X_train.shape), dtype=np.float64)
    X_rotated[range(0, len(X_train), 2)] = rotate_X(X_train[range(0, len(X_train), 2)], degree1)
    X_rotated[range(1, len(X_train), 2)] = rotate_X(X_train[range(1, len(X_train), 2)], degree2)

    y_rotated = np.zeros((y_train.shape), dtype=np.float64)
    y_rotated[range(0, len(y_train), 2)] = rotate_y(y_train[range(0, len(y_train), 2)], degree1, n_point)
    y_rotated[range(1, len(y_train), 2)] = rotate_y(y_train[range(1, len(y_train), 2)], degree2, n_point)   

    X_rotated = np.concatenate((X_train, X_rotated), axis=0)
    y_rotated = np.concatenate((y_train, y_rotated), axis=0)
    
    X_contrast = X_rotated.copy()
    X_contrast[range(1, len(X_rotated), 2)] = reduce_contrast(X_rotated[range(1, len(X_rotated), 2)], percent)
    
    return X_contrast.reshape(-1,96,96,1), y_rotated, X_val.reshape(-1,96,96,1), y_val

# Model 1 with Locally Connected Layer

In [7]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(x_train, y_train, n_point):

    n_conv_layer = 6
    conv_filter1 = 8
    conv_filter2 = 32
    conv_filter3 = 64
    conv_filter4 = 96
    conv_filter5 = 128
    conv_filter6 = 256
    
    n_dense_layer = 3
    dense_neuron1 = 256
    dense_neuron2 = 128
    dense_neuron3 = 96
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-LC-Y-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # lc layer 4
    model.add(LocallyConnected2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # lc layer 5
    model.add(LocallyConnected2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # lc layer 6
    model.add(LocallyConnected2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description

In [5]:
# load data
X = np.load('X.npy')
y = np.load('y.npy')

In [ ]:
# create data
X_model1, y_model1, X_model1_val, y_model1_val = prep_data(X, y, points = [0, 1, 2, 3, 20, 21, 28, 29], val_pct=0)
X_model2, y_model2, X_model2_val, y_model2_val = prep_data(X, y, points=[4,5,6,7,8,9,10,11,12,13,14,15,16, \
                                                                         17,18,19,22,23,24,25,26,27], val_pct=0)

In [ ]:
model, description = create_model(X_model1, y_model1, 8, 'PReLU')
print(description)

# fit model
result = model.fit(X_model1, y_model1, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


In [ ]:
model, description = create_model(X_model1, y_model1, 8, 'PReLU')
print(description)

# fit model
result = model.fit(X_model2, y_model2, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


# Model 2

In [8]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(x_train, y_train, n_point):

    n_conv_layer = 6
    conv_filter1 = 8
    conv_filter2 = 64
    conv_filter3 = 96
    conv_filter4 = 128
    conv_filter5 = 256
    conv_filter6 = 324
    
    n_dense_layer = 3
    dense_neuron1 = 256
    dense_neuron2 = 96
    dense_neuron3 = 64
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-LC-N-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # conv layer 4
    model.add(Conv2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # conv layer 5
    model.add(Conv2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 6
    model.add(Conv2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description

In [ ]:
model, description = create_model(X_model1, y_model1, 8, 'PReLU')
print(description)

# fit model
result = model.fit(X_model1, y_model1, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


In [ ]:
model, description = create_model(X_model1, y_model1, 8, 'PReLU')
print(description)

# fit model
result = model.fit(X_model2, y_model2, batch_size=64, epochs=20, verbose=2, validation_split=0.05)

#get the highest validation accuracy of the training epochs
validation_RMSE = np.amin(result.history['val_rmse'])
print('Best validation RMSE:', validation_RMSE)


# Model 3

In [ ]:
# define RMSE
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# create model - use random search
def create_model(x_train, y_train, n_point):

    n_conv_layer = 8
    conv_filter1 = 28
    conv_filter2 = 64
    conv_filter3 = 64
    conv_filter4 = 64
    conv_filter5 = 128
    conv_filter6 = 128
    conv_filter7 = 256
    conv_filter8 = 256
    
    n_dense_layer = 3
    dense_neuron1 = 256
    dense_neuron2 = 96
    dense_neuron3 = 64
     
    description = 'ConvLayer-{}-Filter-{}-{}-{}-{}-{}-{}-{}-{}-LC-N-DenseLayer-{}-Neuron-{}-{}-{}-Activation-PReLU'.format( \
            n_conv_layer, conv_filter1, conv_filter2, conv_filter3, conv_filter4, conv_filter5, conv_filter6, conv_filter7, conv_filter8, \
            n_dense_layer, dense_neuron1, dense_neuron2, dense_neuron3)
    
    
    # define model
    model = Sequential()
    
    # conv layer 1
    model.add(Conv2D(conv_filter1, (3, 3), strides=(1,1), padding='same', \
                     input_shape=(96,96,1), data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    # conv layer 2
    model.add(Conv2D(conv_filter2, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 3
    model.add(Conv2D(conv_filter3, (3, 3), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(PReLU())
        
    # conv layer 4
    model.add(LocallyConnected2D(conv_filter4, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
            
    # conv layer 5
    model.add(LocallyConnected2D(conv_filter5, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    # conv layer 6
    model.add(LocallyConnected2D(conv_filter6, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    
    # conv layer 7
    model.add(LocallyConnected2D(conv_filter7, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    
    # conv layer 8
    model.add(LocallyConnected2D(conv_filter8, (3, 3), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(PReLU())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    
    
    model.add(Flatten())
    
    # fully connected layer 1
    model.add(Dense(dense_neuron1))
    model.add(PReLU())
    
    # fully connected layer 2
    model.add(Dense(dense_neuron2))
    model.add(PReLU())
        
    # fully connected layer 3
    model.add(Dense(dense_neuron3))
    model.add(PReLU())   
    
    # output layer
    model.add(Dense(n_point))


    # compile model
    model.compile(loss='mean_squared_error', metrics=[rmse], optimizer='adam')

    return model, description